# Imports

In [1]:
# common / KNN imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import scipy.spatial.distance
from collections import Counter
plt.set_cmap('gray')

# Aufgaben

## A KNN

#### How many distances you need to calculate if you have 60,000 samples in the trainingset for 50 samples?
Man muss 3.000.000 Abstände berechnen.
#### How many distances do you need to calculate if you have n samples im the trainingset?
Man muss immer n * 50 Abstände berechnen, d.h. bei 3 Samples müsste man 150 Abstände berechnen, bei 90.000 dann 4.500.000

## B KNN

### What is the error rate of KNN on the test set?

In [2]:
def knn(k):
    dists = scipy.spatial.distance.cdist(x_train_sample, x_test_sample, metric='euclid')
    idx_nearest = np.argpartition(dists, k, axis=0)[:k]
    idx_nearest = np.swapaxes(idx_nearest, 0, 1)
    #print(idx_nearest)

    closest_points_labels = y_train_sample[idx_nearest]
    predictions = []
    
    for index, predicted_labels in enumerate(closest_points_labels):
        predictions.append(np.bincount(predicted_labels).argmax())
    return predictions

In [32]:
def print_error_rate(predictions):
    
    correct = 0
    
    correct_labels = (y_test_sample)
    
    numbers_count = Counter(correct_labels)
    
    numbers_correct = dict.fromkeys(numbers_count.keys(), 0)
    
    #numbers_error_rate = dict.fromkeys(numbers_count.keys(), None)
    
    
    n = correct_labels.shape[0]
    
    for index, prediction in enumerate(predictions):
        if prediction == correct_labels[index]:
            correct += 1
            numbers_correct[prediction] += 1
        
    error_rate = 1- (correct / n)
    
    
    for key in sorted(numbers_count):
        count = numbers_count[key]
        correct = numbers_correct[key]
        error = 1 - numbers_correct[key] / numbers_count[key]
        print("Label:", key)
        print("  ", correct, "out of", count, "recognized correctly | Errorrate:", error*100, "%")
        
    print("\n"+"Resulting errorrate:", error_rate*100, "%")

In [4]:
#Load dataset
mnist_data = tf.keras.datasets.mnist.load_data()

# Get Training set
train = mnist_data[0]
X_train, Y_train = train[0], train[1]

# Get Test set
test = mnist_data[1]
X_test, Y_test = test[0], test[1]

n_train = X_train.shape[0]
n_test = X_test.shape[0]

m = X_train.shape[1]**2

X_train = X_train.reshape([n_train, m])
X_test = X_test.reshape([n_test, m])

# Create train subset
idx = np.random.randint(n_train, size=10000)
x_train_sample = X_train[idx]
y_train_sample = Y_train[idx]

# Create test subset
idx = np.random.randint(n_test, size=30)
x_test_sample = X_test[idx]
y_test_sample = Y_test[idx]

In [5]:
predictions2 = knn(2)
predictions4 = knn(4)
predictions8 = knn(8)

print(list(y_test_sample), "\n")

print("-------------------------------------")
print("  2NN  ")
print("-------------------------------------")
print("Predictions:")
print(predictions2, "\n")
print_error_rate(predictions2)

print("-------------------------------------")
print("  4NN  ")
print("-------------------------------------")
print("Predictions:")
print(predictions4, "\n")
print_error_rate(predictions4)

print("-------------------------------------")
print("  8NN  ")
print("-------------------------------------")
print("Predictions:")
print(predictions8, "\n")
print_error_rate(predictions8)


[8, 9, 8, 0, 1, 9, 1, 2, 4, 2, 3, 6, 7, 9, 7, 7, 2, 4, 2, 1, 8, 4, 8, 8, 5, 5, 1, 0, 6, 4] 

-------------------------------------
  2NN  
-------------------------------------
Predictions:
[8, 9, 8, 0, 1, 9, 1, 2, 4, 2, 3, 6, 7, 9, 7, 7, 2, 4, 2, 1, 8, 4, 8, 8, 4, 5, 1, 0, 6, 4] 

Label: 0
   2 out of 2 recognized correctly | Errorrate: 0.0 %
Label: 1
   4 out of 4 recognized correctly | Errorrate: 0.0 %
Label: 2
   4 out of 4 recognized correctly | Errorrate: 0.0 %
Label: 3
   1 out of 1 recognized correctly | Errorrate: 0.0 %
Label: 4
   4 out of 4 recognized correctly | Errorrate: 0.0 %
Label: 5
   1 out of 2 recognized correctly | Errorrate: 50.0 %
Label: 6
   2 out of 2 recognized correctly | Errorrate: 0.0 %
Label: 7
   3 out of 3 recognized correctly | Errorrate: 0.0 %
Label: 8
   5 out of 5 recognized correctly | Errorrate: 0.0 %
Label: 9
   3 out of 3 recognized correctly | Errorrate: 0.0 %

Resulting errorrate: 0.033333333333333326
-------------------------------------
  4NN

##### What is the error rate of KNN on the test set?
Die Errorrate beim ersten Testen (mit k=2, train = 10.000, test = 30) war 6%.  
Beim testen mit neuen subsets gab es auch Errorraten zwischen 3 - 16%
##### What is the error rate for each label (number)?
Die Errorraten beim ersten Testen (mit k=2) waren:
* 0 - 0.0%
* 1 - 0.0%
* 2 - 0.0%
* 3 - 0.0%
* 4 - 33.0% (2/3)
* 5 - 20.0% (4/5)
* 6 - 0.0%
* 7 - 0.0%
* 8 - 0.0%
* 9 - 0.0%  
  
Beim testen mit anderen Subsets fällt auf, dass es bei den Zahlen 0-2 tendenziell niedrigere Fehlerquoten gibt als beim Rest.
##### How does the choice of k influence the result?
Beim ersten Testen hat sich die Errorrate mit zunehmendem k verbessert:

* k = 2: 6%
* k = 4: 3%
* k = 8: 3%  
  
Beim Testen mit anderen Subsets ist aufgefallen, dass k sich nicht immer auf die Errorquote auswirkt, aber k=4 tendenziell niedrigere Errorraten für unsere Test- und Traingrößen hatte.

## C K-means MNIST

In [7]:
# load subset
n_sample = 1000
n = X_train.shape[0]

idx = np.random.randint(n, size=n_sample)
x_sample = X_train[idx]
y_sample = Y_train[idx]

k = 10
q = 30
_tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
embedding = _tsne.fit_transform(x_sample)


[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.023s...
[t-SNE] Computed neighbors for 1000 samples in 1.448s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 593.085839
[t-SNE] KL divergence after 250 iterations with early exaggeration: 67.988281
[t-SNE] KL divergence after 300 iterations: 1.379070


In [25]:
# init centroids
idx_centroid = np.random.randint(n_sample, size=k)
centroids = x_sample[idx_centroid]
    
# calc distances
dists = scipy.spatial.distance.cdist(x_sample, centroids, metric='euclid')
    
# find nearest centroids
labels = np.argmin(dists, axis=1)

In [26]:
def kmeans_train(q):
# train model for q iterations
    for j in range(q):
        
        # calc distances
        dists = scipy.spatial.distance.cdist(x_sample, centroids, metric='euclid')
        
        # label according to nearest centroids
        labels = np.argmin(dists, axis=1)
        
        # get new centroids
        centroid_list = [x_sample[labels == i].mean(0) for i in range(k)]
        centroids = np.stack(centroid_list)
        
kmeans_train(1000)

In [37]:
# print results

for i in range(k):
    
    size = y_sample[labels == i].shape[0]
    bincount = np.bincount(y_sample[labels == i])
    mode = bincount.argmax()
    correct = bincount[mode]
    percentage = correct / size
    
    print('group', i)
    print("-------------------------------------")
    print("label", mode, "(size:", size, ")")
    print("correct", correct)
    print("size", size)
    print("Percentage of majority class:", percentage*100, "%", "\n")

group 0
-------------------------------------
label 9 (size: 287 )
correct 53
size 287
Percentage of majority class: 18.466898954703833 % 

group 1
-------------------------------------
label 0 (size: 17 )
correct 6
size 17
Percentage of majority class: 35.294117647058826 % 

group 2
-------------------------------------
label 4 (size: 99 )
correct 42
size 99
Percentage of majority class: 42.42424242424242 % 

group 3
-------------------------------------
label 0 (size: 45 )
correct 38
size 45
Percentage of majority class: 84.44444444444444 % 

group 4
-------------------------------------
label 7 (size: 28 )
correct 12
size 28
Percentage of majority class: 42.857142857142854 % 

group 5
-------------------------------------
label 0 (size: 59 )
correct 13
size 59
Percentage of majority class: 22.033898305084744 % 

group 6
-------------------------------------
label 7 (size: 164 )
correct 48
size 164
Percentage of majority class: 29.268292682926827 % 

group 7
-------------------------

#### Do this for 10, 100, 1000 iterations
##### What is the majority class of each cluster?
##### What is the percentage of the majority class in each cluster?
##### Does each number have a cluster?
##### If not, which hasn’t?


## D MNIST understanding

## E KNN and K-means CIFAR

## F Linear regression BOSTON

## G Logistik regression MNIST

## H Feedback